In [1]:
import graphlab

In [2]:
song_data = graphlab.SFrame('song_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1484501742.log


This non-commercial license of GraphLab Create for academic use is assigned to rongjin@indiana.edu and will expire on January 13, 2018.


In [3]:
song_data.head(3)

Columns:
	user_id	str
	song_id	str
	listen_count	int
	title	str
	artist	str
	song	str

Rows: 3

Data:
+-------------------------------+--------------------+--------------+
|            user_id            |      song_id       | listen_count |
+-------------------------------+--------------------+--------------+
| b80344d063b5ccb3212f76538f... | SOAKIMP12A8C130995 |      1       |
| b80344d063b5ccb3212f76538f... | SOBBMDR12A8C13253B |      2       |
| b80344d063b5ccb3212f76538f... | SOBXHDL12A81C204C0 |      1       |
+-------------------------------+--------------------+--------------+
+-----------------+---------------+-------------------------------+
|      title      |     artist    |              song             |
+-----------------+---------------+-------------------------------+
|     The Cove    |  Jack Johnson |    The Cove - Jack Johnson    |
| Entre Dos Aguas | Paco De Lucia | Entre Dos Aguas - Paco De ... |
|     Stronger    |   Kanye West  |     Stronger - Kanye West     |
+-----------------+---------------+-------------------------------+
[3 rows x 6 columns]

In [4]:
Kanye_West_rows = song_data[song_data['artist']=='Kanye West']
len(Kanye_West_rows['user_id'].unique())

2522

In [5]:
Foo_Fighters_rows = song_data[song_data['artist']=='Foo Fighters']
len(Foo_Fighters_rows['user_id'].unique())

2055

In [6]:
Taylor_Swift_rows = song_data[song_data['artist']=='Taylor Swift']
len(Taylor_Swift_rows['user_id'].unique())

3246

In [7]:
Lady_GaGa_rows = song_data[song_data['artist']=='Lady GaGa']
len(Lady_GaGa_rows['user_id'].unique())

2928

In [9]:
popular_song = song_data.groupby(key_columns='artist', \
                                operations={'total_count': graphlab.aggregate.SUM('listen_count')})

In [11]:
popular_song.sort('total_count',ascending=False)

artist,total_count
Kings Of Leon,43218
Dwight Yoakam,40619
Björk,38889
Coldplay,35362
Florence + The Machine,33387
Justin Bieber,29715
Alliance Ethnik,26689
OneRepublic,25754
Train,25402
The Black Keys,22184


In [4]:
graphlab.canvas.set_target('ipynb')

In [5]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [17]:
personalized_model = graphlab.item_similarity_recommender.create(train_data,
                                                                user_id='user_id',
                                                                item_id='song')

Recsys training: model = item_similarity

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 1.94s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.06827s

In [18]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [20]:
personalized_recommendation = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 10685.8

recommendations finished on 2000/10000 queries. users per second: 12700.4

recommendations finished on 3000/10000 queries. users per second: 13959.5

recommendations finished on 4000/10000 queries. users per second: 14409.2

recommendations finished on 5000/10000 queries. users per second: 14699.9

recommendations finished on 6000/10000 queries. users per second: 15004.3

recommendations finished on 7000/10000 queries. users per second: 15093.3

recommendations finished on 8000/10000 queries. users per second: 15116.4

recommendations finished on 9000/10000 queries. users per second: 14921.3

recommendations finished on 10000/10000 queries. users per second: 14834

In [21]:
personalized_recommendation.head(3)

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.0194504536115,1
c067c22072a17d33310d7223d7b79f819e48cf42 ...,Grind With Me (ExplicitVersion) - Pretty Ricky ...,0.0459424376488,1
f6c596a519698c97f1591ad89f540d76f6a04f1a ...,Hey_ Soul Sister - Train,0.0248946808279,1


In [24]:
popular_recommend = personalized_recommendation.groupby(key_columns='song', \
                                    operations={'count': graphlab.aggregate.COUNT()})

In [25]:
popular_recommend.sort('count',ascending=False)

song,count
Undo - Björk,433
Secrets - OneRepublic,387
Revelry - Kings Of Leon,228
You're The One - DwightYoakam ...,159
Fireflies - CharttraxxKaraoke ...,122
Sehr kosmisch - Harmonia,98
Horn Concerto No. 4 in Eflat K495: II. Romance ...,94
Hey_ Soul Sister - Train,93
OMG - Usher featuringwill.i.am ...,62
Dog Days Are Over (RadioEdit) - Florence + The ...,43
